In [1]:
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
import os

plotly.offline.init_notebook_mode(connected=True)

gdp_per_capita = pd.read_csv("../resources/gdp-per-capita-in-us-dollar-world-bank.csv").rename(columns ={"GDP per capita (constant 2010 US$) (constant 2010 US$)":"gdp_per_capita"})
gdp_urbanization = pd.read_csv("../resources/urbanization-vs-gdp.csv")
world_gdp = pd.read_csv("../resources/world-gdp-over-the-last-two-millennia.csv")



In [2]:
gdp_per_capita_2017 = gdp_per_capita.loc[gdp_per_capita["Year"] == 2017, :]
gdp_per_capita_2017.head()


,Entity,Code,Year,gdp_per_capita
15,Afghanistan,AFG,2017,618.332790
53,Albania,ALB,2017,4868.197347
111,Algeria,DZA,2017,4825.178585
174,Andorra,AND,2017,43942.938187
212,Angola,AGO,2017,3484.599515


In [3]:
data = [go.Choropleth(
    locations = gdp_per_capita_2017['Code'],
    z = gdp_per_capita_2017['gdp_per_capita'],
    text = gdp_per_capita_2017['Entity'],
    colorscale = [
        [0, "rgb(5, 10, 172)"],
        [0.35, "rgb(40, 60, 190)"],
        [0.5, "rgb(70, 100, 245)"],
        [0.6, "rgb(90, 120, 245)"],
        [0.7, "rgb(106, 137, 247)"],
        [1, "rgb(220, 220, 220)"]
    ],
    autocolorscale = False,
    reversescale = True,
    marker = go.choropleth.Marker(
        line = go.choropleth.marker.Line(
            color = 'rgb(180,180,180)',
            width = 0.5
        )),
    colorbar = go.choropleth.ColorBar(
        tickprefix = '$',
        title = 'GDP<br>Billions US$'),
)]

layout = go.Layout(
    title = go.layout.Title(
        text = '2017 Global GDP'
    ),
    geo = go.layout.Geo(
        showframe = False,
        showcoastlines = False,
        projection = go.layout.geo.Projection(
            type = 'equirectangular'
        )
    ),
    annotations = [go.layout.Annotation(
        x = 0.55,
        y = 0.1,
        xref = 'paper',
        yref = 'paper',
        text = 'Source: Gautum Gupta',
        showarrow = False
    )]
)

plotly.offline.iplot({
    "data": data,
    "layout": layout
})



In [3]:
gdp_urbanization.head()

,Entity,Code,Year,Share of population living in urban areas (%),GDP per capita (2011 int-$) ($),Total population (Gapminder)
0,Afghanistan,AFG,1800,NaN,NaN,3280000.0
1,Afghanistan,AFG,1820,NaN,NaN,3280000.0
2,Afghanistan,AFG,1870,NaN,NaN,4207000.0
3,Afghanistan,AFG,1913,NaN,NaN,5730000.0
4,Afghanistan,AFG,1950,6.0,2392.0,8151455.0


In [4]:
world_gdp.head()

,Entity,Code,Year,(int.-$)
0,World,OWID_WRL,1,1.827410e+11
1,World,OWID_WRL,1000,2.101450e+11
2,World,OWID_WRL,1500,4.305270e+11
3,World,OWID_WRL,1600,5.744690e+11
4,World,OWID_WRL,1700,6.433230e+11


In [5]:
A = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
A.head()

,COUNTRY,GDP (BILLIONS),CODE
0,Afghanistan,21.71,AFG
1,Albania,13.40,ALB
2,Algeria,227.80,DZA
3,American Samoa,0.75,ASM
4,Andorra,4.80,AND


In [75]:
import plotly
import plotly.graph_objs as go

plotly.offline.iplot({
    "data": [go.Scatter(x=[1, 2, 3, 4], y=[4, 3, 2, 1])],
    "layout": go.Layout(title="hello world")
})